In [2]:
pip install PyPortfolioOpt

  Obtaining dependency information for PyPortfolioOpt from https://files.pythonhosted.org/packages/e3/7d/4930adde3743e4d3bf4e30a19af14c3d9f622e8e7b4e6bb7dfb68f98ce8f/pyportfolioopt-1.5.5-py3-none-any.whl.metadata
  Obtaining dependency information for cvxpy<2.0.0,>=1.1.19 from https://files.pythonhosted.org/packages/38/85/5f46779431dd978f62a1f5bd64df05d9a1418cc50d7294e98bf445e31068/cvxpy-1.4.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for osqp>=0.6.2 from https://files.pythonhosted.org/packages/5d/9a/6bcf0d1ada180ee36d2a813bf75473f5246884688a3f5e710ef755c1f35e/osqp-0.6.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for ecos>=2 from https://files.pythonhosted.org/packages/67/1f/165ca12f4b3de6bd3fe8b16695013d28e88808effeff37858427ae56f449/ecos-2.0.13-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for clarabel>=0.5.0 from https://files.pythonhosted.org/packages/a8/6b/0f04f31c1528ab1f6bbc9e6ff83d019386941db4c9e0a7d

In [1]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np


# Define the stock tickers and historical data
stock_tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
start_date = '2020-01-01'
end_date = '2021-01-01'

# Fetch historical stock data
stock_data = pd.DataFrame({ticker: yf.download(ticker, start=start_date, end=end_date)['Adj Close'] for ticker in stock_tickers})

# Calculate expected returns and sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(stock_data)
cov_matrix = risk_models.sample_cov(stock_data)

# Optimize for maximum Sharpe ratio
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe()

# Clean weights
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# Calculate expected return, volatility, and the Sharpe ratio
expected_return = ef.portfolio_performance(verbose=True)

# Get the discrete allocation of each stock for a $10000 investment
latest_prices = get_latest_prices(stock_data)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
OrderedDict([('AAPL', 0.0), ('GOOGL', 0.0), ('MSFT', 0.0), ('AMZN', 0.0), ('TSLA', 1.0)])
Expected annual return: 720.1%
Annual volatility: 89.5%
Sharpe Ratio: 8.03
Discrete allocation: {'TSLA': 42}
Funds remaining: $120.62


In [10]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd
import yfinance as yf
import numpy as np


# Define the stock tickers and historical data
stock_tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
start_year = 2017
end_year = 2021

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

# Function to calculate portfolio returns for a given year
def calculate_portfolio_returns(weights, year):
    # Fetch historical data for each stock
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    # Calculate portfolio returns
    portfolio_returns = (stock_data.pct_change() * weights).sum(axis=1).dropna()

    # Calculate portfolio cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    return cumulative_returns.iloc[-1] - 1  # Return the total return for the year

# Initialize empty dictionary to store annual returns
annual_returns = {}

# Loop through each year
for year in range(start_year, end_year):
    # Fetch historical stock data
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    # Calculate expected returns and sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(stock_data)
    cov_matrix = risk_models.sample_cov(stock_data)

    # Optimize for maximum Sharpe ratio
    ef = EfficientFrontier(mu, cov_matrix)
    weights = ef.max_sharpe()

    # Calculate portfolio returns for the year using the optimal weights
    annual_returns[year] = calculate_portfolio_returns(weights, year)

# Print annual returns
for year, return_value in annual_returns.items():
    print(f"Return for {year}: {return_value:.2%}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [9]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

# Function to fetch historical stock price data
def fetch_stock_data(symbols, start_date, end_date):
    data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
    return data

# Function to calculate expected returns and covariance matrix
def calculate_expected_returns_and_covariance(data):
    returns = data.pct_change().dropna()
    expected_returns = returns.mean()
    cov_matrix = returns.cov()
    return expected_returns * 100, cov_matrix * 100

# Function to compute portfolio metrics
def compute_portfolio_metrics(weights, expected_returns, cov_matrix):
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_std_dev

# Function to compute negative Sharpe ratio
def negative_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate):
    portfolio_return, portfolio_std_dev = compute_portfolio_metrics(weights, expected_returns, cov_matrix)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return -sharpe_ratio

# Function to optimize portfolio weights
def optimize_portfolio(expected_returns, cov_matrix, risk_free_rate):
    num_assets = len(expected_returns)
    initial_weights = np.ones(num_assets) / num_assets
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 1) for asset in range(num_assets))
    result = minimize(negative_sharpe_ratio, initial_weights, args=(expected_returns, cov_matrix, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Function to simulate portfolio performance over years
def simulate_portfolio_performance(symbols, start_date, end_date, num_years, risk_free_rate):
    portfolio_performance = []
    for year in range(num_years):
        # Fetch historical data
        end_year = int(end_date.split('-')[0]) - year
        start_year = end_year - 1
        data = fetch_stock_data(symbols, f"{start_year}-01-01", f"{end_year}-12-31")

        # Calculate expected returns and covariance matrix
        expected_returns, cov_matrix = calculate_expected_returns_and_covariance(data)

        # Optimize portfolio weights
        optimal_weights = optimize_portfolio(expected_returns, cov_matrix, risk_free_rate)

        # Validate weights
        if not np.isclose(np.sum(optimal_weights), 1.0):
            print(f"Warning: Sum of weights is not 1 for year {end_year}")

        # Validate returns
        if np.isnan(expected_returns).any():
            print(f"Warning: NaN values found in expected returns for year {end_year}")

        # Compute expected return for the year
        expected_return, _ = compute_portfolio_metrics(optimal_weights, expected_returns, cov_matrix)

        # Fetch actual returns for the current year
        actual_returns = fetch_stock_data(symbols, f"{end_year}-01-01", f"{end_year}-12-31").pct_change().mean() * 100

        # Compute actual return of the portfolio for the year
        actual_return = np.sum(actual_returns * optimal_weights)

        # Append results to list
        portfolio_performance.append({'Year': end_year, 'Optimal Weights': optimal_weights,
                                      'Expected Return (%)': expected_return,
                                      'Actual Return (%)': actual_return})

    return pd.DataFrame(portfolio_performance)

# Define parameters
symbols = ['DRREDDY.NS', 'TCS.NS', 'WIPRO.NS', 'HCLTECH.NS']  # Example portfolio
start_date = '2018-01-01'  # Start date for historical data
end_date = '2022-01-01'  # End date for historical data
num_years = 5  # Number of years to simulate
risk_free_rate = 0.07  # Assumed risk-free rate

# Simulate portfolio performance
portfolio_performance = simulate_portfolio_performance(symbols, start_date, end_date, num_years, risk_free_rate)

# Display portfolio performance
print(portfolio_performance)


[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
[*********************100%%**********************]  4 of 4 completed
   Year                                    Optimal Weights  \
0  2022                               [0.0, 1.0, 0.0, 0.0]   
1  2021  [0.022681675592918442, 0.25602031164822747, 3....   
2  2020  [0.5433988113678516, 0.45660118863214844, 0.0,...   
3  2019  [2.0816681711721685e-17, 5.551115123125783e-17...   


In [22]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd
import yfinance as yf
import numpy as np


# Define the stock tickers and historical data
stock_tickers = ['DRREDDY.NS', 'HCLTECH.NS', 'TCS.NS', 'WIPRO.NS']
start_year = 2015
end_year = 2024

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

# Function to calculate portfolio returns for a given year
def calculate_portfolio_metrics(weights, year):
    # Fetch historical data for each stock
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    # Calculate portfolio returns
    portfolio_returns = (stock_data.pct_change() * weights).sum(axis=1).dropna()

    # Calculate portfolio cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    # Calculate total return for the year
    total_return = cumulative_returns.iloc[-1] - 1

    return total_return, weights  # Return the total return and weights for the year

# Initialize empty dictionary to store annual returns and weights
annual_returns_and_weights = {}

# Loop through each year
for year in range(start_year, end_year):
    # Fetch historical stock data
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    # Calculate expected returns and sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(stock_data)
    cov_matrix = risk_models.sample_cov(stock_data)

    # Optimize for maximum Sharpe ratio
    ef = EfficientFrontier(mu, cov_matrix)
    weights = ef.max_sharpe(risk_free_rate=-0.5)  # Assuming a risk-free rate of 2%


    # Calculate portfolio metrics for the year using the optimal weights
    annual_returns_and_weights[year] = calculate_portfolio_metrics(weights, year)

# Print annual returns and weights
for year, (return_value, weights) in annual_returns_and_weights.items():
    print(f"Year: {year}")
    print(f"Return: {return_value:.2%}")
    print("Weights:")
    for ticker, weight in weights.items():
        print(f"{ticker}: {weight:.2%}")
    print()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******